In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='300d', interval='1d')
    data.reset_index(inplace=True, drop=True)
    return data
# Get the data
data = get_data('BTC-USD')

[*********************100%%**********************]  1 of 1 completed


In [24]:
# Calculate Bollinger Bands using pandas_ta
data.ta.bbands(length=10, std=1.5, append=True)

# Add the upper and lower bands to the DataFrame
data['Upper Band'] = data['BBU_10_1.5']
data['Lower Band'] = data['BBL_10_1.5']

def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

In [25]:
data

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA
19,16863.472656,16884.021484,16790.283203,16836.736328,16836.736328,13692758566,16522.863641,16675.555273,16828.246906,1.831323,1.027799,16828.246906,16522.863641,16735.449121
20,16836.472656,16991.994141,16716.421875,16951.968750,16951.968750,14413662913,16501.865780,16699.034766,16896.203751,2.361442,1.141414,16896.203751,16501.865780,16743.292969
21,16952.117188,16975.017578,16914.191406,16955.078125,16955.078125,7714767174,16526.789650,16739.285352,16951.781053,2.538886,1.007758,16951.781053,16526.789650,16753.148047
22,16954.146484,17091.144531,16924.050781,17091.144531,17091.144531,9768827914,16526.553665,16784.165625,17041.777585,3.069702,1.095816,17041.777585,16526.553665,16785.721289
23,17093.992188,17389.957031,17093.992188,17196.554688,17196.554688,18624736866,16544.778301,16843.562500,17142.346699,3.547755,1.090714,17142.346699,16544.778301,16800.233789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,27971.677734,28102.169922,27740.662109,27935.089844,27935.089844,7916875290,27008.997727,27588.998437,28168.999148,4.204580,0.798354,28168.999148,27008.997727,27105.326074
296,27934.472656,27989.470703,27302.562500,27583.677734,27583.677734,12007668568,27183.923881,27656.194141,28128.464401,3.415295,0.423226,28128.464401,27183.923881,27123.954102
297,27589.201172,27715.847656,27301.654297,27391.019531,27391.019531,9973350678,27340.392422,27698.504492,28056.616562,2.585786,0.070686,28056.616562,27340.392422,27136.904687
298,27392.076172,27474.115234,26561.099609,26873.320312,26873.320312,13648094333,27102.278649,27587.461523,28072.644398,3.517416,-0.235951,28072.644398,27102.278649,27152.189063


In [26]:
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['Close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['Close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Trend'] = check_candles(data, 7, 'SMA')

In [27]:
data

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend
19,16863.472656,16884.021484,16790.283203,16836.736328,16836.736328,13692758566,16522.863641,16675.555273,16828.246906,1.831323,1.027799,16828.246906,16522.863641,16735.449121,0
20,16836.472656,16991.994141,16716.421875,16951.968750,16951.968750,14413662913,16501.865780,16699.034766,16896.203751,2.361442,1.141414,16896.203751,16501.865780,16743.292969,0
21,16952.117188,16975.017578,16914.191406,16955.078125,16955.078125,7714767174,16526.789650,16739.285352,16951.781053,2.538886,1.007758,16951.781053,16526.789650,16753.148047,0
22,16954.146484,17091.144531,16924.050781,17091.144531,17091.144531,9768827914,16526.553665,16784.165625,17041.777585,3.069702,1.095816,17041.777585,16526.553665,16785.721289,0
23,17093.992188,17389.957031,17093.992188,17196.554688,17196.554688,18624736866,16544.778301,16843.562500,17142.346699,3.547755,1.090714,17142.346699,16544.778301,16800.233789,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,27971.677734,28102.169922,27740.662109,27935.089844,27935.089844,7916875290,27008.997727,27588.998437,28168.999148,4.204580,0.798354,28168.999148,27008.997727,27105.326074,2
296,27934.472656,27989.470703,27302.562500,27583.677734,27583.677734,12007668568,27183.923881,27656.194141,28128.464401,3.415295,0.423226,28128.464401,27183.923881,27123.954102,2
297,27589.201172,27715.847656,27301.654297,27391.019531,27391.019531,9973350678,27340.392422,27698.504492,28056.616562,2.585786,0.070686,28056.616562,27340.392422,27136.904687,2
298,27392.076172,27474.115234,26561.099609,26873.320312,26873.320312,13648094333,27102.278649,27587.461523,28072.644398,3.517416,-0.235951,28072.644398,27102.278649,27152.189063,2


## 1 - Entry Based on Bollinger Bands - Candles Crossing

In [28]:
# Check conditions and assign entry values
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & ((data['Open'] < data['Lower Band']) & (data['Close'] > data['Lower Band']))
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & ((data['Open'] > data['Upper Band']) & (data['Close'] < data['Upper Band']))
data.loc[sell_entry_condition, 'entry'] = 1

In [29]:
data[data['entry']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry
202,29907.998047,30434.644531,29777.285156,30342.265625,30342.265625,13384770155,29999.301138,30491.948828,30984.596519,3.231330,0.348083,30984.596519,29999.301138,29952.875879,2,2
228,29704.146484,29987.998047,28946.509766,29151.958984,29151.958984,19212655598,29083.906304,29297.924414,29511.942524,1.460978,0.158988,29511.942524,29083.906304,29666.586426,1,1


In [30]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA'], mode='lines', name='SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Lower Band'], mode='lines', name='Lower Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Upper Band'], mode='lines', name='Upper Band', line=dict(color='blue')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 2 - Entry based on RSI and Bollinger Bands

In [106]:
def add_rsi_column(data):
    # Calculate RSI with a period of 14
    data['RSI'] = ta.rsi(data['Close'])
    return data

data = add_rsi_column(data)

In [117]:
def rsi_signal(data):
    data['RSI Signal'] = 0  # Initialize the signal column with 0

    # Set the signal category to 2 when the price is below the lower Bollinger Band and RSI is below 30
    data.loc[(data['Close'] < data['Lower Band']) & (data['RSI'] < 55), 'RSI Signal'] = 2

    # Set the signal category to 1 when the price is above the upper Bollinger Band and RSI is above 70
    data.loc[(data['Close'] > data['Upper Band']) & (data['RSI'] > 45), 'RSI Signal'] = 1

    return data

In [118]:
data = rsi_signal(data)

In [119]:
data[data["RSI Signal"]!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,shooting_star,RSI,Signal,RSI Signal
34,21085.373047,22692.357422,20919.126953,22676.552734,22676.552734,28799154319,18641.087235,20535.443555,22429.799874,18.449627,1.065128,22429.799874,18641.087235,18734.442871,2,0,0,90.376149,1,1
35,22677.427734,23282.347656,22511.833984,22777.625000,22777.625000,32442278429,19385.569762,21019.716406,22653.863050,15.548703,1.037867,22653.863050,19385.569762,19042.070117,2,0,0,90.581672,1,1
43,23031.449219,23919.890625,22985.070312,23774.566406,23774.566406,27423687259,22507.052546,22978.051563,23449.050579,4.099556,1.345559,23449.050579,22507.052546,21495.234570,2,0,0,85.905490,1,1
46,23137.835938,23764.539062,22877.750000,23723.769531,23723.769531,26683255504,22620.920753,23130.911328,23640.901903,4.409602,1.081244,23640.901903,22620.920753,22267.855273,2,0,0,74.645951,1,1
54,22946.566406,22996.437500,21773.974609,21819.039062,21819.039062,32572572185,22330.587368,23085.459766,23840.332163,6.539808,-0.338831,23840.332163,22330.587368,23039.934961,0,0,0,47.300374,0,2
55,21819.005859,21941.185547,21539.392578,21651.183594,21651.183594,27078406594,21945.582733,22936.649805,23927.716877,8.641777,-0.148526,23927.716877,21945.582733,22983.612891,0,0,0,45.751532,0,2
60,22220.585938,24307.841797,22082.769531,24307.841797,24307.841797,32483312909,21212.986884,22442.984766,23672.982647,10.961090,1.258073,23672.982647,21212.986884,22861.306543,1,0,0,68.253790,0,1
62,23621.283203,24924.041016,23460.755859,24565.601562,24565.601562,41358451255,21059.342428,22659.452344,24259.562260,14.123112,1.095631,24259.562260,21059.342428,22965.269434,0,0,0,66.322376,0,1
69,23946.007812,24103.705078,23007.072266,23198.126953,23198.126953,26811744928,23501.805915,24206.580273,24911.354632,5.822998,-0.215444,24911.354632,23501.805915,23257.173730,2,2,0,50.462747,0,2
70,23200.125000,23210.210938,22861.558594,23175.375000,23175.375000,16100721565,23253.806693,24093.333594,24932.860494,6.968956,-0.046712,24932.860494,23253.806693,23268.159180,0,0,0,50.266914,0,2


In [120]:
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & (data['RSI Signal'] == 2) & (data['Low'] < data['Lower Band'])
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & (data['RSI Signal'] == 1) & (data['High'] > data['Upper Band'])
data.loc[sell_entry_condition, 'entry'] = 1

In [121]:
data[data['entry']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,shooting_star,RSI,Signal,RSI Signal
60,22220.585938,24307.841797,22082.769531,24307.841797,24307.841797,32483312909,21212.986884,22442.984766,23672.982647,10.961090,1.258073,23672.982647,21212.986884,22861.306543,1,1,0,68.253790,0,1
69,23946.007812,24103.705078,23007.072266,23198.126953,23198.126953,26811744928,23501.805915,24206.580273,24911.354632,5.822998,-0.215444,24911.354632,23501.805915,23257.173730,2,2,0,50.462747,0,2
86,22156.406250,24550.837891,21918.199219,24197.533203,24197.533203,49466362688,20149.943045,21870.062891,23590.182737,15.730360,1.176543,23590.182737,20149.943045,22646.330078,1,1,0,60.859961,0,1
123,30394.187500,30411.054688,28669.898438,28822.679688,28822.679688,24571565421,29250.572675,30021.098633,30791.624591,5.133230,-0.277663,30791.624591,29250.572675,29084.347754,2,2,0,51.475497,0,2
162,26871.158203,28193.449219,26802.751953,28085.646484,28085.646484,14545229578,26244.485008,26933.486133,27622.487258,5.116316,1.336109,27622.487258,26244.485008,27029.812207,1,1,0,56.900755,0,1
184,26335.441406,26984.611328,26312.832031,26851.029297,26851.029297,12826986222,25333.348031,26033.908789,26734.469547,5.381910,1.083190,26734.469547,25333.348031,26407.798730,1,1,0,52.729743,0,1
234,29180.019531,30176.796875,29113.814453,29765.492188,29765.492188,17570561357,28899.204811,29261.593555,29623.982298,2.476890,1.195246,29623.982298,28899.204811,29390.958496,1,1,0,54.890441,0,1
255,26102.486328,28089.337891,25912.628906,27727.392578,27727.392578,29368391712,25554.178289,26291.876172,27029.574055,5.611603,1.472970,27029.574055,25554.178289,27326.791504,1,1,0,52.713963,0,1
281,26579.373047,26716.058594,26221.050781,26256.826172,26256.826172,8192867686,26270.425639,26679.198633,27087.971627,3.064357,-0.016634,27087.971627,26270.425639,26298.147754,2,2,0,45.516063,0,2
298,27392.076172,27474.115234,26561.099609,26873.320312,26873.320312,13648094333,27102.278649,27587.461523,28072.644398,3.517416,-0.235951,28072.644398,27102.278649,27152.189063,2,2,0,46.370587,0,2


## 3 - Entry based on a rejection candle next to Bollinger Bands

In [94]:
def identify_shooting_star(data):
    # Create a new column for shooting star
    data['shooting_star'] = data.apply(lambda row: 2 if (
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 1 if (
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 0, axis=1)

    return data

In [95]:
data = identify_shooting_star(data)

In [96]:
data[data['shooting_star']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,shooting_star
47,23720.824219,24167.210938,23468.595703,23471.871094,23471.871094,32066936882,22664.018595,23184.655273,23705.291952,4.491218,0.775831,23705.291952,22664.018595,22445.970117,2,0,1
67,24437.417969,24472.339844,23644.318359,24188.843750,24188.843750,30199996781,22406.345355,23894.908984,25383.472614,12.459253,0.598731,25383.472614,22406.345355,23238.951270,2,0,2
87,24201.765625,26514.716797,24081.183594,24746.074219,24746.074219,54622230164,19955.613286,22109.335352,24263.057417,19.482468,1.112135,24263.057417,19955.613286,22674.191602,0,0,1
137,28680.494141,29259.533203,28178.388672,29006.308594,29006.308594,19122972518,27824.766153,28736.539648,29648.313144,6.345743,0.647936,29648.313144,27824.766153,28904.100391,0,0,2
201,30507.150391,31460.052734,29892.226562,29909.337891,29909.337891,21129219509,30032.948585,30526.538672,31020.128758,3.233842,-0.125216,31020.128758,30032.948585,29761.296387,2,0,1
